In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization,Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from IPython.display import SVG,Image

In [2]:
# Getting an overview of the dataset
import os
test="E:\\Download\\Facial_Expression_Recognition_dataset\\test\\"
train="E:\Download\Facial_Expression_Recognition_dataset\\train\\"
#lst=str (os.listdir("E:\\Download\\Facial_Expression_Recognition_dataset\\test\\" ))+"Images"
#lst=list(lst)
#print(lst)
#for i in lst:
    #print(str(len(os.listdir("E:\\Download\\Facial_Expression_Recognition_dataset\\test\\"+i)))+"Images")

In [3]:
for expression in os.listdir("E:\\Download\\Facial_Expression_Recognition_dataset\\test\\"):
    print(str(len(os.listdir("E:\\Download\\Facial_Expression_Recognition_dataset\\train\\"+expression)))+" "+expression+' images')
#we can serr here we have balanced data set except disgust

3995 angry images
436 disgust images
4097 fear images
7215 happy images
4965 neutral images
4830 sad images
3171 surprise images


In [4]:
# Training and Validation batches

train_datagen=ImageDataGenerator(horizontal_flip=True)                # Imagedatagenerator performs the standardisation of the 
                                                                       # image and datagen is its object
train_set=train_datagen.flow_from_directory(train,                     # flow_from_directory generates batches
                                           target_size=(48,48),
                                           batch_size=64,
                                           color_mode ='grayscale',
                                           class_mode='categorical',
                                           shuffle=True
                                           )

test_datagen=ImageDataGenerator(horizontal_flip=True)                                                                                      
test_set=train_datagen.flow_from_directory(test,                     
                                           target_size=(48,48),
                                           batch_size=64,
                                           color_mode='grayscale',
                                           class_mode='categorical',
                                           shuffle=True
                                           )
test_set

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [5]:
# making models
model=Sequential()

In [6]:
# adding various CNN layers
#CNN - 1
model.add(Conv2D(512,(3,3),input_shape=(48,48,1)))             # what is passed and why
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))     
model.add(BatchNormalization())          # Extra layers for better accuracy
model.add(Dropout(0.25))                 # to avoid overfitting

#CNN - 2
model.add(Conv2D(512,(3,3) ))             
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))     
model.add(BatchNormalization())          
model.add(Dropout(0.25))       

#CNN - 3

model.add(Conv2D(512,(3,3) ))             
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))     
model.add(BatchNormalization())          
model.add(Dropout(0.25)) 

#CNN - 4
model.add(Conv2D(512,(3,3) ))             
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))     
model.add(BatchNormalization())          
model.add(Dropout(0.25))        


In [7]:
#Flattening the resultant matrix
model.add(Flatten())           

In [8]:
# adding fully connected dense layers
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
          
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
          
model.add(Dense(7))
model.add(Activation('softmax'))

In [9]:
# compiling and optimizing the model
model.compile(optimizer=Adam(lr=0.0005),loss='categorical_crossentropy',metrics=['accuracy'])
#model.summary()

In [10]:
epochs=5
batch_size=64
steps_per_epoch=train_set.n//train_set.batch_size
validation_steps=test_set.n//test_set.batch_size

In [14]:
# Fitting / making the model
model.fit(
    x=train_set,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=test_set,
    validation_steps=validation_steps
    )
model.save('Emotion_Detection.h5')

Train for 448 steps, validate for 112 steps
Epoch 1/5
448/448 [==============================] - 7663s 17s/step - loss: 2.0272 - accuracy: 0.2740 - val_loss: 1.6828 - val_accuracy: 0.3696
Epoch 2/5
448/448 [==============================] - 5960s 13s/step - loss: 1.5700 - accuracy: 0.4118 - val_loss: 1.3944 - val_accuracy: 0.4544
Epoch 3/5
448/448 [==============================] - 10995s 25s/step - loss: 1.3816 - accuracy: 0.4781 - val_loss: 1.2854 - val_accuracy: 0.5078
Epoch 4/5
448/448 [==============================] - 5922s 13s/step - loss: 1.2640 - accuracy: 0.5217 - val_loss: 1.3042 - val_accuracy: 0.4953
Epoch 5/5
448/448 [==============================] - 5828s 13s/step - loss: 1.1935 - accuracy: 0.5441 - val_loss: 1.2011 - val_accuracy: 0.5540


In [2]:

# Loading the model
from  tensorflow.keras.models import load_model
model=load_model('Emotion_detection.h5')


In [12]:
# classification of the labels

from tensorflow.keras.preprocessing import image

#test_img=image.load_img("E:\\Download\\Facial_Expression_Recognition_dataset\\test\\fear\\PrivateTest_4002000.jpg",color_mode = "grayscale",target_size=(48,48))

# Prediction
def predictor(test_img):
    
    test_img=image.img_to_array(test_img)
    test_img=np.expand_dims(test_img,axis=0)
    
    #Applyig prediction to the model
    result = model.predict(test_img)       

    #NOTE:Here result is a numpy array which contais the probability of every class: The result class will have the higher probalbilty.

    # getting the index of the maximum probability
    final_class_index=result.argmax()

    #print(final_class_index)

    emotions=['Angry','Disgust','Fear','Happy','Neutral','Sad','Surprise']
    print(emotions[final_class_index])
    return emotions[final_class_index]

# Predictions through live camera

#Starting the webcam
cap=cv2.VideoCapture(0)

while(True):
    ret,frame=cap.read()
    faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')   # ML model to recognise faces
    
    #Coverting the coloured frame to grayscale image
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #Finding the coordinates of  the faces
    faces = faceCascade.detectMultiScale(frame, scaleFactor=1.2, minNeighbors=5, minSize=(20, 20))
    
    for(x,y,w,h) in faces:
        #Cropping the face area only
        crop=gray[y:y+h,x:x+h]
        roi=cv2.resize(crop,(48,48))
        path="picture.jpg"     #Saving the cropped face into current folder
        cv2.imwrite(path,roi)
        
        #Calling the predictor function which will return the emotion in the frame
        emotion=predictor(roi)
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        #cv2.putText(frame,emotion,(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 0, 0),2)
        font = cv2.FONT_HERSHEY_PLAIN
        cv2.putText(frame,emotion , (x, y + 30), font, 3,(255,0,0), 3)
        
    cv2.imshow('frame',frame)
    if(cv2.waitKey(1) & 0xFF==ord('q')):
       # cv2.imwrite("myimage.jpg",frame)
        break
        
cap.release()  
cv2.destroyAllWindows()

Sad
Sad
Sad
Sad
Happy
Sad
Sad
Sad
Neutral
Sad
Sad
Neutral
Sad
Neutral
Sad
Neutral
Neutral
Neutral
Sad
Neutral
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Neutral
Neutral
Happy
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Surprise
Surprise
Surprise
Surprise
Surprise
Surprise
Surprise
Surprise
Surprise
Surprise
Surprise
Surprise
Neutral
Neutral
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Sad
Sad
Sad
Sad
Sad
Sad
Happy
Sad
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
Happy
